In [ ]:
#깃허브 테스트
#rebase test

In [2]:
import numpy as np
import nibabel as nib   # 의료 및 뇌 영상 파일 형식에 대한 읽기/쓰기를 위한 모듈

import itk              # 2D 및 3D 이미지 시각화 라이브러리
import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns   # Matplotlib를 기반으로 하는 파이썬 데이터 시각화 라이브러리
sns.set_style('darkgrid')
from scipy import ndimage

image_data = nib.load("/Users/leebyeongju/Desktop/segmentation visualize/data/BraTS20_Training_001_t1.nii").get_fdata()
output_data = nib.load("/Users/leebyeongju/Desktop/segmentation visualize/data/output_0.nii.gz").get_fdata()
mask_data= nib.load("/Users/leebyeongju/Desktop/segmentation visualize/data/BraTS20_Training_001_seg.nii").get_fdata()

# 데이터를 시각화하는 함수를 정의합니다.
def explore_3d_image(layer_x,layer_y,layer_z):
    
    plt.figure(figsize=(10,15))
    ax1= plt.subplot(1,3,1)
    ax1.imshow(image_data[:,:,layer_z],cmap='gray')
    ax1.imshow(mask_data[:,:,layer_z],alpha=0.4)
    plt.axis('off')
    ax2=plt.subplot(1,3,2)
    ax2.imshow(ndimage.rotate(image_data[layer_x,:,:],90),cmap='gray')
    ax2.imshow(ndimage.rotate(mask_data[layer_x,:,:],90),alpha=0.4)
    ax3=plt.subplot(1,3,3)
    ax3.imshow(ndimage.rotate(image_data[:,layer_y,:],90),cmap='gray')
    ax3.imshow(ndimage.rotate(mask_data[:,layer_y,:],90),alpha=0.4)
    plt.axis('off')
    plt.show()
    
# Run the ipywidgets interact() function to explore the data
interact(explore_3d_image, layer_x =(0, image_data.shape[0]-1),layer_y= (0, image_data.shape[1]-1), layer_z=(0, image_data.shape[2]-1))

interactive(children=(IntSlider(value=119, description='layer_x', max=239), IntSlider(value=119, description='…

<function __main__.explore_3d_image(layer_x, layer_y, layer_z)>

In [2]:
import numpy as np
import nibabel as nib   # 의료 및 뇌 영상 파일 형식에 대한 읽기/쓰기를 위한 모듈
import vtk
import itk              # 2D 및 3D 이미지 시각화 라이브러리
import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline
import seaborn as sns   # Matplotlib를 기반으로 하는 파이썬 데이터 시각화 라이브러리
sns.set_style('darkgrid')
from scipy import ndimage
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QGridLayout, QSlider, QOpenGLWidget
from PyQt5.QtCore import Qt
from vtk.qt.QVTKRenderWindowInteractor import QVTKRenderWindowInteractor
from matplotlib.backends.backend_qt5agg import FigureCanvas as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
from IPython.display import display

image_data = nib.load("/Users/leebyeongju/Desktop/segmentation visualize/data/BraTS20_Training_001_t1.nii").get_fdata()
mask_data = nib.load("/Users/leebyeongju/Desktop/segmentation visualize/data/output_0.nii.gz").get_fdata()

class VisualizerWidget(QOpenGLWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.vis = o3d.visualization.Visualizer()
        

class MyApp(QMainWindow):
    
    def __init__(self):
        super().__init__()
        self.vis_widget = VisualizerWidget()
        self.setWindowTitle('Brain Tumor Segmentation')
        self.setGeometry(200, 100, 1000, 800)
        
        self.slider_z = QSlider(Qt.Horizontal, self)
        self.slider_z.setRange(0, image_data.shape[2]-1)
        self.slider_z.setSingleStep(1)
        
        self.slider_x = QSlider(Qt.Horizontal, self)
        self.slider_x.setRange(0, image_data.shape[0]-1)
        self.slider_x.setSingleStep(1)
        
        self.slider_y = QSlider(Qt.Horizontal, self)
        self.slider_y.setRange(0, image_data.shape[1]-1)
        self.slider_y.setSingleStep(1)
        
        self.main_widget = QWidget()
        self.setCentralWidget(self.main_widget)
        
        c1 = FigureCanvas(Figure())
        c2 = FigureCanvas(Figure())
        c3 = FigureCanvas(Figure())
        
        layout = QGridLayout(self.main_widget)
        layout.addWidget(self.slider_z, 0, 0)
        layout.addWidget(self.slider_x, 2, 0)
        layout.addWidget(self.slider_y, 0, 1)
        layout.addWidget(c1, 1, 0)
        layout.addWidget(c2, 3, 0)
        layout.addWidget(c3, 1, 1)
        layout.addWidget(self.vis_widget,3,1)
        
        tumor = o3d.io.read_triangle_mesh("/Users/leebyeongju/Desktop/segmentation visualize/data/output/tumor.obj")
        self.vis_widget.vis.add_geometry(tumor)
        
        self.slider_z.valueChanged.connect(self.update_z)
        self.slider_x.valueChanged.connect(self.update_x)
        self.slider_y.valueChanged.connect(self.update_y)
        
        self.ax1 = c1.figure.subplots()
        self.ax1.imshow(image_data[:,:,100],cmap='gray')
        self.ax1.imshow(mask_data[:,:,100],alpha=0.3)
        self.ax1.axis('off')
        
        self.ax2=c2.figure.subplots()
        self.ax2.imshow(ndimage.rotate(image_data[100,:,:],90),cmap='gray')
        self.ax2.imshow(ndimage.rotate(mask_data[100,:,:],90),alpha=0.3)
        self.ax2.axis('off')
        
        self.ax3=c3.figure.subplots()
        self.ax3.imshow(ndimage.rotate(image_data[:,100,:],90),cmap='gray')
        self.ax3.imshow(ndimage.rotate(mask_data[:,100,:],90),alpha=0.3)
        self.ax3.axis('off')
      
        self.show()
        
    def update_z(self):
        self.ax1.clear()
        self.ax1.imshow(image_data[:,:,self.slider_z.value()],cmap='gray')
        self.ax1.imshow(mask_data[:,:,self.slider_z.value()],alpha=0.3)
        self.ax1.axis('off')
        self.ax1.figure.canvas.draw()
        
    def update_x(self):
        self.ax2.clear()
        self.ax2.imshow(ndimage.rotate(image_data[self.slider_x.value(),:,:],90),cmap='gray')
        self.ax2.imshow(ndimage.rotate(mask_data[self.slider_x.value(),:,:],90),alpha=0.3)
        self.ax2.axis('off')
        self.ax2.figure.canvas.draw()
        
    def update_y(self):
        self.ax3.clear()
        self.ax3.imshow(ndimage.rotate(image_data[:,self.slider_y.value(),:],90),cmap='gray')
        self.ax3.imshow(ndimage.rotate(mask_data[:,self.slider_y.value(),:],90),alpha=0.3)
        self.ax3.axis('off')
        self.ax3.figure.canvas.draw()
     
if __name__ == '__main__':
  app = QApplication(sys.argv)
  ex = MyApp()
  sys.exit(app.exec_())
  
  

ModuleNotFoundError: No module named 'nibabel'

In [1]:
import pyqtgraph.opengl as gl
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout
import sys
import numpy as np

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()

        # Create a mesh
        verts = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0]])
        faces = np.array([[0, 1, 2], [1, 2, 3]])
        colors = np.array([[1, 0, 0, 1], [0, 1, 0, 1]])
        meshdata = gl.MeshData(vertexes=verts, faces=faces, faceColors=colors)

        # Create a GLViewWidget and add the mesh to it
        self.view = gl.GLViewWidget()
        self.view.addItem(gl.GLMeshItem(meshdata=meshdata, smooth=False))

        # Set up the layout
        layout = QVBoxLayout()
        layout.addWidget(self.view)
        self.setLayout(layout)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())


/Users/leebyeongju/opt/anaconda3/lib/python3.9/site-packages/pyqtgraph/colors/palette.py:1: RuntimeWarning: PyQtGraph supports Qt version >= 5.15, but 5.9.7 detected.
  from ..Qt import QtGui


SystemExit: 0

/Users/leebyeongju/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
pip install nibabel

     |████████████████████████████████| 3.3 MB 6.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyqtgraph.opengl as gl
from PyQt5.QtGui import QBrush, QColor
from PyQt5.QtWidgets import QApplication, QGraphicsScene, QGraphicsView, QVBoxLayout, QWidget
import sys
import pyqtgraph

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()

        # Create a GLViewWidget
        self.view = gl.GLViewWidget()
        
        bgcolor = self.view.setBackgroundColor(pyqtgraph.mkColor('w'))

        # Set up the QGraphicsScene and save the returned value
        scene = QGraphicsScene()
        scene.setBackgroundBrush(QBrush(QColor(0, 0, 0, 0)))
        self.view.setCameraPosition(distance=50, elevation=20, azimuth=120)


        # Load the .obj file
        # Load the .obj file
        with open('/Users/leebyeongju/Desktop/segmentation_GUI/data/output/t2.obj', 'r') as f:
            vertices = []
            faces = []
            for line in f:
                if line.startswith('v '):
                    vertex = list(map(float, line[2:].strip().split(' ')))
                    vertices.append(vertex)
                elif line.startswith('f '):
                    face = line[2:].strip().split(' ')
                    # Ignore additional information (such as texture and normal indices) after vertex indices
                    face_indices = [int(vertex.split('//')[0]) for vertex in face]
                    faces.append(face_indices)
            meshdata = gl.MeshData(vertexes=vertices, faces=faces)


        # Create a GLMeshItem and add it to the GLViewWidget
        meshitem = gl.GLMeshItem(meshdata=meshdata, smooth=True, shader='normalColor')
        self.view.addItem(meshitem)
        self.view.setCameraPosition(distance=300, elevation=30, azimuth=30)
        # Set up the layout
        layout = QVBoxLayout()
        layout.addWidget(self.view)
        self.setLayout(layout)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())


/Users/leebyeongju/opt/anaconda3/lib/python3.9/site-packages/pyqtgraph/colors/palette.py:1: RuntimeWarning: PyQtGraph supports Qt version >= 5.15, but 5.9.7 detected.
  from ..Qt import QtGui
/Users/leebyeongju/opt/anaconda3/lib/python3.9/site-packages/pyqtgraph/opengl/GLViewWidget.py:259: RuntimeWarning: 
Traceback (most recent call last):
  File "/Users/leebyeongju/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/leebyeongju/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/leebyeongju/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/leebyeongju/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/Users/leebyeongju/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677,

Error while drawing item <pyqtgraph.opengl.items.GLMeshItem.GLMeshItem object at 0x7ff40221ef70>.
Error while drawing item <pyqtgraph.opengl.items.GLMeshItem.GLMeshItem object at 0x7ff40221ef70>.
Error while drawing item <pyqtgraph.opengl.items.GLMeshItem.GLMeshItem object at 0x7ff40221ef70>.
Error while drawing item <pyqtgraph.opengl.items.GLMeshItem.GLMeshItem object at 0x7ff40221ef70>.
Error while drawing item <pyqtgraph.opengl.items.GLMeshItem.GLMeshItem object at 0x7ff40221ef70>.
Error while drawing item <pyqtgraph.opengl.items.GLMeshItem.GLMeshItem object at 0x7ff40221ef70>.
Error while drawing item <pyqtgraph.opengl.items.GLMeshItem.GLMeshItem object at 0x7ff40221ef70>.
Error while drawing item <pyqtgraph.opengl.items.GLMeshItem.GLMeshItem object at 0x7ff40221ef70>.
Error while drawing item <pyqtgraph.opengl.items.GLMeshItem.GLMeshItem object at 0x7ff40221ef70>.
Error while drawing item <pyqtgraph.opengl.items.GLMeshItem.GLMeshItem object at 0x7ff40221ef70>.
Error while drawing 

SystemExit: 0

/Users/leebyeongju/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
pip install PyOpenGL

     |████████████████████████████████| 2.4 MB 7.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
